In [ ]:
#Dependencies
import requests
import pandas as pd
import numpy as np
import json
import time
import matplotlib.pyplot as plt
from scipy.stats import linregress
from config import api_key
import seaborn as sns


In [ ]:
team_url = f"http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/standings.json?api_key={api_key}"

team_data = requests.get(team_url).json()
def extract_team_data(data):
    teams = []
    if 'teams' in data:
        for team in data['teams']:
            team_id = team['id']
            team_name = team['name']
            teams.append({'id': team_id, 'name': team_name})
    if 'divisions' in data:
        for division in data['divisions']:
            teams += extract_team_data(division)
    if 'conferences' in data:
        for conference in data['conferences']:
            teams += extract_team_data(conference)
    return teams

# Extract team id and team name from the team_data dictionary
teams = extract_team_data(team_data)
teams_info=pd.DataFrame(teams)
teams_id= teams_info["id"]
time.sleep(1)

In [ ]:
# Initialize
team_name=[]
team_field_goals=[]
team_id=[]
team_data={}
team_offensive_rebounds=[]
team_turnovers=[]
team_free_throws_att=[]
team_possession=[]
team_o_rtg=[]
team_d_rtg=[]
team_net_rtg=[]
field_goals_made=[]
three_points_made=[]
free_throws_made=[]
ow_defensive_rebounds=[]
opp_defensive_rebounds=[]
ow_points=[]
opp_points=[]
i=0
# Loop through each team ID and retrieve their statistics from the API
for i in range(len(teams_id)):
    team_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/teams/{teams_id[i]}/statistics.json?api_key={api_key}'
    team_data = requests.get(team_url).json()
    # Extract various statistics for the each team
    field_goals_att=team_data['own_record']['total']['field_goals_att']
    offensive_rebounds=team_data['own_record']['total']['offensive_rebounds']
    turnovers= team_data['own_record']['total']['turnovers']
    free_throws_att= team_data['own_record']['total']['free_throws_att']
    
    team_totals_dict = team_data['own_record']['total']
    opp_vs_team_totals_dict = team_data['opponents']['total']

    team_stat_keys = ['games_played', 'minutes', 'field_goals_made', 'field_goals_att',
                      'three_points_made', 'three_points_att', 'three_points_pct',
                      'free_throws_made', 'free_throws_att', 'offensive_rebounds',
                      'defensive_rebounds', 'rebounds', 'assists', 'turnovers',
                      'steals', 'blocks', 'points']
    team_stats = {k: v for k, v in team_totals_dict.items() if k in team_stat_keys}
    opp_vs_team_stats = {k: v for k, v in opp_vs_team_totals_dict.items() if k in team_stat_keys}

    #team stats calculations
    possession = 0.96*(field_goals_att + turnovers + (0.44*free_throws_att) -offensive_rebounds)
    o_rtg = 100 * (team_stats['points']) / (possession)
    d_rtg = 100 * (opp_vs_team_stats['points']) / (possession)
    net_rtg = (o_rtg) - (d_rtg)

    #Append the extracted statistics to teams' lists
    team_name.append(team_data['name'])
    team_id.append(team_data['id']) 
    team_field_goals.append(field_goals_att)
    field_goals_made.append(team_stats["field_goals_made"])
    team_offensive_rebounds.append(offensive_rebounds)
    team_turnovers.append(turnovers)
    team_free_throws_att.append(free_throws_att)
    team_possession.append(possession)
    team_o_rtg.append(o_rtg)
    team_d_rtg.append(d_rtg)
    team_net_rtg.append(net_rtg)
    three_points_made.append(team_stats["three_points_made"])
    free_throws_made.append(team_stats["free_throws_made"])
    ow_defensive_rebounds.append(team_stats["defensive_rebounds"])
    opp_defensive_rebounds.append(opp_vs_team_stats["defensive_rebounds"])
    ow_points.append(team_stats['points'])
    opp_points.append(opp_vs_team_stats['points'])


    # Pause execution for 0.5 second to avoid overloading the API
    time.sleep(1)  

# Create Teams DataFrame
teams_df=pd.DataFrame({
    'Team Name' : team_name,
    #'Team ID': team_id,
    'Team Possession': team_possession,
    'team_o_rtg': team_o_rtg,
    'team_d_rtg': team_d_rtg,
    'team_net_rtg': team_net_rtg ,
        
    'Team Field Goals Attempts':team_field_goals,
    'Team Field Goals Made':field_goals_made,
    'Team Offensive Rebounds': team_offensive_rebounds,
    'Team Defensive Rebounds': ow_defensive_rebounds,
    'Opp Defensive Rebounds': opp_defensive_rebounds,
    'Team Turnovers': team_turnovers,
    'Team Free throws Attemps': team_free_throws_att,
    'Team Free throws Made': free_throws_made,
    'Team Three Points Made':three_points_made,
    'Team Points': ow_points,
    'Opp Ponits': opp_points
   

})
teams_df

In [ ]:
standing_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/standings.json?api_key={api_key}'
standing_data = requests.get(standing_url).json()
aa= standing_data["conferences"]
Teams_name=[]
standing_rank=[]
conferences2=[]
win=[]
n=i=j=0

for n in range(len(aa)):
    zz= standing_data["conferences"][n]['divisions']
    
    
    for i in range(len(zz)):
        bb=standing_data["conferences"][n]['divisions'][i]['teams']
        

        for j in range(len(bb)):
            Teams_name.append(standing_data["conferences"][n]['divisions'][i]['teams'][j]['name'])
            standing_rank.append(standing_data["conferences"][n]['divisions'][i]['teams'][j]['calc_rank']['conf_rank'])
            win.append(standing_data["conferences"][n]['divisions'][i]['teams'][j]['wins'])
            conferences2.append(standing_data["conferences"][n]['alias'])
    #b=pd.DataFrame[b]
    standing_df=pd.DataFrame({"Team Name": Teams_name,
                'Wins': win,
                'Conference Ranking': standing_rank,
                'Conferences':conferences2

   })
standing_df

In [ ]:
last_team_df=pd.merge(standing_df,teams_df,on="Team Name")
last_team_df=last_team_df.sort_values('Conference Ranking')
last_team_df

In [ ]:
#wins vs defense
plt.scatter(last_team_df["Wins"], last_team_df["team_d_rtg"], edgecolor="Blue", alpha=1)

plt.xlabel("Wins")
plt.ylabel("Teams Defensive Rating")


plt.grid(True)
x_values=last_team_df["Wins"]
y_values=last_team_df["team_d_rtg"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
r_values= x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,r_values,"r-")
plt.annotate(line_eq,(30,114),fontsize=15,color="red")
print("The r-values is: "+ str(rvalue))
plt.show()

In [ ]:
#wins vs offense
plt.scatter(last_team_df["Wins"], last_team_df["team_o_rtg"], edgecolor="Blue", alpha=1)

plt.xlabel("Wins")
plt.ylabel("Offensive Rating")


plt.grid(True)
x_values=last_team_df["Wins"]
y_values=last_team_df["team_o_rtg"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
r_values= x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,r_values,"r-")
plt.annotate(line_eq,(30,114),fontsize=15,color="red")
print("The r-values is: "+ str(rvalue))
plt.show()

In [ ]:
plt.scatter(last_team_df["Wins"], last_team_df["team_net_rtg"], edgecolor="Blue", alpha=1)

plt.xlabel("Wins")
plt.ylabel("Teams Net Rating")


#plt.ylim(-30, 50)

plt.grid(True)
x_values=last_team_df["Wins"]
y_values=last_team_df["team_net_rtg"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
r_values= x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,r_values,"r-")
plt.annotate(line_eq,(20,0),fontsize=15,color="red")
print("The r-values is: "+ str(rvalue))
plt.show()